Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# English Premier League Teams and Stadiums
This notebook uses a property graph containing the teams that took part in the 2019/20 English Premier League season. The graph includes information about the teams, their stadiums and the cities where they play. This notebook is based on the EPL-Gremlin notebook but contains some differences. The queries provided are expressed using openCypher.

## A note on basic terminology

There are a few commonly used terms used when working with graph databases. The three main ones being node or vertex, edge or relationship, and property. Different communites tend to have some subset of these they use more often. The Apache TinkerPop Gremlin documentation for example uses the terms Vertex and Edge exclusively whereas users of openCypher more commonly use Node and Relationship. The examples presented in this notebook follow those same conventions.  One other area where the terminology differs is that edge labels in Gremlin are commonly referred to as a type of relationship in openCypher.

## Seeing a visual representation of your query

The results of any openCypher query that returns a path (`MATCH p= ... RETURN p`) or simple list of nodes can be displayed visually. When such queries are run you will see a `Graph` tab in the query results area alongside the `Console` tab. Specific visualization features will be explained  using examples throughout this notebook. 


### Query visualization hints

Settings can be used to give the visual renderer hints as to how to label the nodes drawn.

You can specify query visualization hints using  `-d`, `-de`, `-l` and `-g` after the `%%oc` or `opencypher` cell "magics". Either one of them can be used to run an openCypher query. The syntax in general is:

`%%opencypher -d $my_vertex_mapping -de $my_edge_mapping -l <max label length> -g <property to group by>`

All of these settings are optional. By default nodes in the visualization display their label text. The parameter `$my_mapping` refers to a Python variable that you create in a different cell that provides a mapping from a node's label to the name of a property you prefer be used to label nodes in the visualization. For example

`my_vertex_mapping = '{"Team":"name","Stadium":"code"}`

By default labels shown in the visualization are truncated at 10 characters and the last 3 of the ten are replaced by an ellipsis. You can override this setting using the `-l` setting. For example:

`%%opencypher -l 20`

Nodes in the visualization are colored (grouped) using their labels by default. You can change this using the `-g` option. For example, to group nodes by the `founded` property:

`%%opencypher -g founded`

Grouping can be completely disabled using the `--ignore-groups` option.

### Adjusting the visualization layout and other settings
You can further adjust many of the visualization settings using the two commands

- `%graph_notebook_vis_options`
- `%%graph_notebook_vis_options`

## Getting help
You can get help at any time using the `--help` option as follows (the cell body needs at least one character in it for `--help` to work). 

`%%opencypher --help`

Several of other the magic commands will accept `--help`. Each of the options described above also has a long form. For example `-d` can also be specified using `--display-options`. Run the cell below to see the full help text for the `%%opencypher` command.


In [ ]:
%%opencypher --help
x

## Check the status of our connection
The three cells below can be used to check the version of the workbench, the current configuration, and the status of the Neptune cluster.

In [ ]:
%graph_notebook_version

#### Using a different endpoint
You can change the endpoint that this notebook is connected to at any time using the `%graph_notebook_host` or `%%graph_notebook_config` commands. Edit and run the cell below whenever you need to change the endpoint you are connected to.

In [ ]:
%graph_notebook_host your-hostname-here

In [ ]:
%graph_notebook_config

#### Neptune endpoint status

In [ ]:
%status

## Create the graph
The cell below creates the EPL 2019.20 property graph. A set of queries that can be run against the data follows in the subsequent cells. A slightly different version of this graph can also be loaded via the `%seed` command. The main difference being that graph is created using Gremlin steps which allocate specific ID values to every vertex and some types and labels are in lower case. When working with openCypher, explicitly accessing a vertex by its ID value is a less used pattern and user defined ID values are not supported when creating nodes and relationships.

In [ ]:
%%oc
CREATE (epl:League { name:"English Premier League",nickname: "EPL"})
CREATE (arsenal:Team {name:"Arsenal", fullName:"Arsenal F. C.", nickname:"The Gunners", founded:1886})
CREATE (chelsea:Team {name:"Chelsea", fullName:"Chelsea F.C.", nickname:"The Blues", founded:1905})
CREATE (tottenhamhotspur:Team {name:"Tottenham Hotspur", fullName:"Tottenham Hotspur F.C.", nickname:"Spurs", founded:1882})
CREATE (westhamunited:Team {name:"West Ham United", fullName:"West Ham United F.C.", nickname:"The Hammers", founded:1895})
CREATE (liverpool:Team {name:"Liverpool", fullName:"Liverpool F.C.", nickname:"The Reds", founded:1892})
CREATE (everton:Team {name:"Everton", fullName:"Everton F.C.", nickname:"The Toffees", founded:1878})
CREATE (manchesterunited:Team {name:"Manchester United", fullName:"Manchester United F.C.", nickname:"The Red Devils", founded:1878})
CREATE (manchestercity:Team {name:"Manchester City", fullName:"Manchester City F.C", nickname:"The Citizens", founded:1880})
CREATE (wolverhamptonwanderers:Team {name:"Wolverhampton Wanderers", fullName:"Wolverhampton Wanderers F.C", nickname:"Wolves", founded:1877})
CREATE (burnley:Team {name:"Burnley", fullName:"Burnley F.C", nickname:"The Clarets", founded:1882})
CREATE (sheffieldunited:Team {name:"Sheffield United", fullName:"Sheffield United F.C", nickname:"The Blades", founded:1889})
CREATE (crystalpalace:Team {name:"Crystal Palace", fullName:"Crystal Palace F.C", nickname:"Eagles", founded:1905})
CREATE (southampton:Team {name:"Southampton", fullName:"Southampton F.C", nickname:"The Saints", founded:1885})
CREATE (newcastleunited:Team {name:"Newcastle United", fullName:"Newcastle United F.C", nickname:"The Magpies", founded:1892})
CREATE (brightonandhovealbion:Team {name:"Brighton and Hove Albion", fullName:"Brighton and Hove Albion F.C", nickname:"Seagulls", founded:1901})
CREATE (watford:Team {name:"Watford", fullName:"Watford F.C.", nickname:"Hornets", founded:1898})
CREATE (bournemouth:Team {name:"Bournemouth", fullName:"Bournemouth F.C.", nickname:"The Cherries", founded:1899})
CREATE (astonvilla:Team {name:"Aston Villa", fullName:"Aston Villa F.C.", nickname:"The Villans", founded:1897})
CREATE (leicestercity:Team {name:"Leicester City", fullName:"Leicester City F.C.", nickname:"The Foxes", founded:1884})
CREATE (norwichcity:Team {name:"Norwich City", fullName:"Norwich City F.C.", nickname:"The Canaries", founded:1902})
CREATE (theemirates:Stadium {name:"The Emirates", opened:2006, capacity:60704})
CREATE (stamfordbridge:Stadium {name:"Stamford Bridge", opened:1877, capacity:40834})
CREATE (tottenhamhotspurstadium:Stadium {name:"Tottenham Hotspur Stadium", opened:2019, capacity:62214})
CREATE (londonstadium:Stadium {name:"London Stadium", opened:2016, capacity:60000})
CREATE (anfield:Stadium {name:"Anfield", opened:1884, capacity:53394})
CREATE (goodisonpark:Stadium {name:"Goodison Park", opened:1892, capacity:39414})
CREATE (oldtrafford:Stadium {name:"Old Trafford", opened:1910, capacity:75643})
CREATE (etihadstadium:Stadium {name:"Etihad Stadium", opened:2003, capacity:55107})
CREATE (molineuxstadium:Stadium {name:"Molineux Stadium", opened:1889, capacity:32050})
CREATE (turfmoor:Stadium {name:"Turf Moor", opened:1883, capacity:21944})
CREATE (bramalllane:Stadium {name:"Bramall Lane", opened:1855, capacity:32125})
CREATE (selhurstparkstadium:Stadium {name:"Selhurst Park Stadium", opened:1924, capacity:25486})
CREATE (stmarysstadium:Stadium {name:"St. Mary's Stadium", opened:2001, capacity:32384})
CREATE (stjamespark:Stadium {name:"St. James' Park", opened:1880, capacity:52305})
CREATE (americanexpresscommunitystadium:Stadium {name:"American Express Community Stadium", opened:2011, capacity:30750})
CREATE (vicarageroad:Stadium {name:"Vicarage Road", opened:1922, capacity:22200})
CREATE (vitalitystadium:Stadium {name:"Vitality Stadium", opened:1910, capacity:11364})
CREATE (villapark:Stadium {name:"Villa Park", opened:1897, capacity:42095})
CREATE (kingpowerstadium:Stadium {name:"King Power Stadium", opened:2002, capacity:32261})
CREATE (carrowroadstadium:Stadium {name:"Carrow Road Stadium", opened:1935, capacity:27359})
CREATE (citylondon:City {name:"London"})
CREATE (cityliverpool:City {name:"Liverpool"})
CREATE (citymanchester:City {name:"Manchester"})
CREATE (citywolverhampton:City {name:"Wolverhampton"})
CREATE (cityleicester:City {name:"Leicester"})
CREATE (citybrighton:City {name:"Brighton"})
CREATE (citysouthampton:City {name:"Southampton"})
CREATE (citynewcastle:City {name:"Newcastle"})
CREATE (citysheffield:City {name:"Sheffield"})
CREATE (citybirmingham:City {name:"Birmingham"})
CREATE (citywatford:City {name:"Watford"})
CREATE (citynorwich:City {name:"Norwich"})
CREATE (citybournemouth:City {name:"Bournemouth"})
CREATE (cityburnley:City {name:"Burnley"})
CREATE (arsenal)-[:STADIUM]->(theemirates)-[:CITY]->(citylondon)
CREATE (chelsea)-[:STADIUM]->(stamfordbridge)-[:CITY]->(citylondon)
CREATE (tottenhamhotspur)-[:STADIUM]->(tottenhamhotspurstadium)-[:CITY]->(citylondon)
CREATE (westhamunited)-[:STADIUM]->(londonstadium)-[:CITY]->(citylondon)
CREATE (crystalpalace)-[:STADIUM]->(selhurstparkstadium)-[:CITY]->(citylondon)
CREATE (liverpool)-[:STADIUM]->(anfield)-[:CITY]->(cityliverpool)
CREATE (everton)-[:STADIUM]->(goodisonpark)-[:CITY]->(cityliverpool)
CREATE (manchesterunited)-[:STADIUM]->(oldtrafford)-[:CITY]->(citymanchester)
CREATE (manchestercity)-[:STADIUM]->(etihadstadium)-[:CITY]->(citymanchester)
CREATE (wolverhamptonwanderers)-[:STADIUM]->(molineuxstadium)-[:CITY]->(citywolverhampton)
CREATE (leicestercity)-[:STADIUM]->(kingpowerstadium)-[:CITY]->(cityleicester)
CREATE (brightonandhovealbion)-[:STADIUM]->(americanexpresscommunitystadium)-[:CITY]->(citybrighton)
CREATE (southampton)-[:STADIUM]->(stmarysstadium)-[:CITY]->(citysouthampton)
CREATE (newcastleunited)-[:STADIUM]->(stjamespark)-[:CITY]->(citynewcastle)
CREATE (sheffieldunited)-[:STADIUM]->(bramalllane)-[:CITY]->(citysheffield)
CREATE (astonvilla)-[:STADIUM]->(villapark)-[:CITY]->(citybirmingham)
CREATE (watford)-[:STADIUM]->(vicarageroad)-[:CITY]->(citywatford)
CREATE (norwichcity)-[:STADIUM]->(carrowroadstadium)-[:CITY]->(citynorwich)
CREATE (bournemouth)-[:STADIUM]->(vitalitystadium)-[:CITY]->(citybournemouth)
CREATE (burnley)-[:STADIUM]->(turfmoor)-[:CITY]->(cityburnley)
CREATE (arsenal)-[:CURRENT_LEAGUE]->(epl)
CREATE (chelsea)-[:CURRENT_LEAGUE]->(epl)
CREATE (tottenhamhotspur)-[:CURRENT_LEAGUE]->(epl)
CREATE (westhamunited)-[:CURRENT_LEAGUE]->(epl)
CREATE (liverpool)-[:CURRENT_LEAGUE]->(epl)
CREATE (everton)-[:CURRENT_LEAGUE]->(epl)
CREATE (manchesterunited)-[:CURRENT_LEAGUE]->(epl)
CREATE (manchestercity)-[:CURRENT_LEAGUE]->(epl)
CREATE (wolverhamptonwanderers)-[:CURRENT_LEAGUE]->(epl)
CREATE (burnley)-[:CURRENT_LEAGUE]->(epl)
CREATE (sheffieldunited)-[:CURRENT_LEAGUE]->(epl)
CREATE (crystalpalace)-[:CURRENT_LEAGUE]->(epl)
CREATE (southampton)-[:CURRENT_LEAGUE]->(epl)
CREATE (newcastleunited)-[:CURRENT_LEAGUE]->(epl)
CREATE (brightonandhovealbion)-[:CURRENT_LEAGUE]->(epl)
CREATE (watford)-[:CURRENT_LEAGUE]->(epl)
CREATE (bournemouth)-[:CURRENT_LEAGUE]->(epl)
CREATE (astonvilla)-[:CURRENT_LEAGUE]->(epl)
CREATE (leicestercity)-[:CURRENT_LEAGUE]->(epl)
CREATE (norwichcity)-[:CURRENT_LEAGUE]->(epl)

### Verify some data was inserted
When the cells below are run you should see the names of the 20 teams in the data set. Note that either `%%oc` or `%%opencypher` may be used to denote that a cell contains an openCypher query.

In [ ]:
%%oc
MATCH (t:Team)
RETURN t.name

### Explore the relationships in the graph

In [ ]:
%%oc
  MATCH (league)<-[e:CURRENT_LEAGUE]-(team)-[s:STADIUM]->(stad)-[c:CITY]->(city)
  RETURN league.name,type(e),team.name,type(s),stad.name,type(c),city.name

In [ ]:
%%oc
MATCH (team)-[e:CURRENT_LEAGUE]->(league)
RETURN team.name AS Team,type(e) AS Type,league.name AS League

### How many teams were in the league that season?

In [ ]:
%%oc
  MATCH (t:Team)
  RETURN count(t) AS count

In [ ]:
%%oc
MATCH (:League {nickname:'EPL'})<-[:CURRENT_LEAGUE]-(t)
RETURN count(t) AS count

### Team info
Find the teams in the graph and their properties. The first query also returns the ID and other information. The second query just returns the properties.

In [ ]:
%%oc  -l20
MATCH (t:Team)
RETURN t AS Team_Info

In [ ]:
%%oc 
MATCH (t:Team)
RETURN properties(t) AS Team_Info

### Team and stadium info

In [ ]:
%%oc 
MATCH  (t:Team)-[:STADIUM]->(stad)-[:CITY]->(city)
RETURN t.name AS Team, stad.name AS Stadium, city.name AS City

### Visualization of the graph

Running the next two cells will create a visualization of the entire EPL graph. The cell immediately below is used to define a mapping between node labels and the property to be used to label nodes in the visualization.

In [ ]:
display_var = '{"Team":"name","City":"name","Stadium":"name","League":"name"}'

The query below creates a path containing the entire graph structure. The `-d` hint is used to enable the mappings defined above. The `-l` hint sets the maximum length of the text that can be displayed in a node. The text will be truncated 3 characters short of that length in cases where there is more text to display. An elipsis `...` is used to indicate that not all of the text was shown.

In [ ]:
%%oc -d $display_var -l20
MATCH p=(lg:League)<--(t:Team)-[:STADIUM]->(s:Stadium)-[:CITY]->(c:City)
RETURN p

### Teams based in London

In [ ]:
%%oc
  MATCH (:City {name:'London'})<-[:CITY]-()<-[:STADIUM]-(t:Team)
  RETURN t.name AS Team

In [ ]:
%%oc  -d $display_var -l20
MATCH p=(:City {name:'London'})<-[:CITY]-()<-[:STADIUM]-(t:Team)
RETURN p

### Stadiums in London

In [ ]:
%%oc
MATCH (:City {name:'London'})<-[:CITY]-(s)
RETURN s.name AS Stadium


In [ ]:
%%oc
MATCH (s)-[:CITY]->(:City {name:'London'})
RETURN s.name AS Stadium

### Teams in London plus their stadiums

In [ ]:
%%oc
MATCH (t:Team)-[:STADIUM]->(s)-[:CITY]->(:City {name:'London'})
RETURN s.name AS Stadium, t.name AS Team

### Finding sub-strings
This query looks for any teams that have the string "ou" in their name.

In [ ]:
%%oc
MATCH (t:Team)
WHERE t.name CONTAINS 'ou'
RETURN t.name AS Team

### Find everything the graph contains related to Arsenal

In [ ]:
%%oc
  MATCH (t:Team {name:'Arsenal'})-->(stadium)-->(city)
  RETURN t.name AS Name,
         t.founded AS Founded,
         t.nickname AS Nickname,
         stadium.name AS Stadium,
         city.name aS City


### How many stadiums are in each city?

In [ ]:
%%oc
  MATCH (s:Stadium)-[:CITY]->(city:City)
  RETURN city.name AS Name, count(*) AS Count 
  ORDER BY Count DESC


### What year did each stadium open?

In [ ]:
%%oc
MATCH (s:Stadium)
RETURN s.name AS Name, s.opened AS Opened
ORDER BY s.opened 

### Stadiums ordered by descending capacity

In [ ]:
%%oc
MATCH (s:Stadium)
RETURN s.name AS Name, s.capacity AS Capacity
ORDER BY s.capacity DESC 

### Number of teams founded in a given year

In [ ]:
%%oc
MATCH (t:Team)
RETURN collect(t.name) AS Name,t.founded AS Founded, count(t) AS Count
ORDER BY Founded

### Teams founded in the same year

In [ ]:
%%oc
  MATCH (a:Team) 
  MATCH (b:Team)
  WHERE a.founded = b.founded AND a <> b
  RETURN a.name AS Name_1, b.name AS Name_2, a.founded AS Founded
  ORDER BY a.founded


In [ ]:
%%oc
  MATCH (a:Team) 
  MATCH (b:Team)
  WHERE a.founded = b.founded AND id(a) > id(b)
  RETURN a.name AS Name_1, b.name AS Name_2, a.founded AS Founded
  ORDER BY a.founded


### Page Width
If you would like Jupyter to maximise the horizontal screen real estate run the cell below.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))